In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

NLTK is a leading platform for building Python programs to work with human language data.
https://www.nltk.org/

In [ ]:
import re
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize
nltk.download('wordnet')
import nltk as nlp

In [ ]:
df = pd.read_csv('../input/trump-tweets/trumptweets.csv',encoding='utf8')
df.head()

In [ ]:
cult_list=[]

for cult in df.content:
    cult=re.sub("[^a-zA-z]"," ",cult)
    cult=cult.lower()
    cult=nltk.word_tokenize(cult)
    lemma=nlp.WordNetLemmatizer()
    cult=[lemma.lemmatize(word) for word in cult]
    cult=" ".join(cult)
    cult_list.append(cult)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

max_features=800
count_vectorizer=CountVectorizer(max_features=max_features,stop_words="english")
sparce_matrix=count_vectorizer.fit_transform(cult_list).toarray()

In [ ]:
sparce_matrix.shape 

In [ ]:
sparce_matrix

In [ ]:
print("Top {} the most used words: {}".format(max_features,count_vectorizer.get_feature_names()))

In [ ]:
data = pd.DataFrame(count_vectorizer.get_feature_names(),columns=["Words"])
data[0:10]

In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt

plt.subplots(figsize=(10,10))
wordcloud=WordCloud(background_color="black",width=1024,height=768).generate(" ".join(data.Words[0:100]))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
X=sparce_matrix[0:20000]
y0=df.favorites[0:20000]

y1=[]
for item in y0:
    y1+=[int(np.log1p(item))]
y=pd.Series(y1)

print(X.shape)
print(y.shape)

In [ ]:
print(y.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,log_loss,precision_score
from sklearn.metrics import roc_auc_score,roc_curve

from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore")

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train,y_train)
y_pred = lgbm_model.predict(X_test)

In [ ]:
print("Accuracy:",accuracy_score(y_test,y_pred))

In [ ]:
df1=df[['favorites','content']].copy()
df1

In [ ]:
df2=df1.sort_values(by=['favorites'],ascending=False).reset_index()
df2

In [ ]:
for i in range(8):
    print('['+str(i+1)+'] '+df2['content'][i])